In [ ]:
# Importing all the necessary libraries
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import os, cv2
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Importing the data

data = pd.read_csv('/content/drive/MyDrive/ham_10000/HAM10000_28.28_L.csv')

Visualising Dataset

In [ ]:
data

,pixel0000,pixel0001,pixel0002,pixel0003,pixel0004,pixel0005,pixel0006,pixel0007,pixel0008,pixel0009,...,pixel0775,pixel0776,pixel0777,pixel0778,pixel0779,pixel0780,pixel0781,pixel0782,pixel0783,label
0,169,171,170,177,181,182,181,185,194,192,...,184,186,185,180,157,140,140,159,165,2
1,19,57,105,140,149,148,144,155,170,170,...,172,175,160,144,114,89,47,18,18,2
2,155,163,161,167,167,172,155,152,165,175,...,163,178,157,166,167,148,141,136,115,2
3,25,71,116,139,136,153,148,161,172,162,...,125,135,138,137,111,71,32,16,16,2
4,129,162,181,196,205,208,205,213,225,224,...,210,197,172,190,195,193,181,147,88,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10010,172,171,173,175,164,187,207,210,208,206,...,210,217,221,209,185,187,192,192,192,0
10011,2,34,108,116,114,119,131,139,139,145,...,173,169,168,168,143,138,83,23,3,0
10012,122,154,162,170,179,197,200,195,202,199,...,221,215,205,187,209,198,187,164,156,0
10013,137,143,141,139,147,152,155,152,155,159,...,172,171,175,183,177,170,169,166,170,0


# Preprocessing dataset

In [ ]:
data.isna().sum().sum()

0

In [ ]:
y = data['label'].copy()
X = data.drop('label', axis=1).copy()

There are 7 possible labels.

From the dataset provider:

0: nv - Melanocytic nevi
1: mel - Melanoma
2: bkl - Benign keratosis-like lesions
3: bcc - Basal cell carcinoma
4: akiec - Actinic keratoses and intraepithelial carcinoma / Bowen's disease
5: vasc - Vascular lesions
6: df - Dermatofibroma

In [ ]:
y.value_counts()

4    6705
6    1113
2    1099
1     514
0     327
5     142
3     115
Name: label, dtype: int64

In [ ]:
label_mapping = {
    0: 'nv',
    1: 'mel',
    2: 'bkl',
    3: 'bcc',
    4: 'akiec',
    5: 'vasc',
    6: 'df'
}

Rescaling

In [ ]:
X = X / 255

X

,pixel0000,pixel0001,pixel0002,pixel0003,pixel0004,pixel0005,pixel0006,pixel0007,pixel0008,pixel0009,...,pixel0774,pixel0775,pixel0776,pixel0777,pixel0778,pixel0779,pixel0780,pixel0781,pixel0782,pixel0783
0,0.662745,0.670588,0.666667,0.694118,0.709804,0.713725,0.709804,0.725490,0.760784,0.752941,...,0.690196,0.721569,0.729412,0.725490,0.705882,0.615686,0.549020,0.549020,0.623529,0.647059
1,0.074510,0.223529,0.411765,0.549020,0.584314,0.580392,0.564706,0.607843,0.666667,0.666667,...,0.709804,0.674510,0.686275,0.627451,0.564706,0.447059,0.349020,0.184314,0.070588,0.070588
2,0.607843,0.639216,0.631373,0.654902,0.654902,0.674510,0.607843,0.596078,0.647059,0.686275,...,0.635294,0.639216,0.698039,0.615686,0.650980,0.654902,0.580392,0.552941,0.533333,0.450980
3,0.098039,0.278431,0.454902,0.545098,0.533333,0.600000,0.580392,0.631373,0.674510,0.635294,...,0.556863,0.490196,0.529412,0.541176,0.537255,0.435294,0.278431,0.125490,0.062745,0.062745
4,0.505882,0.635294,0.709804,0.768627,0.803922,0.815686,0.803922,0.835294,0.882353,0.878431,...,0.827451,0.823529,0.772549,0.674510,0.745098,0.764706,0.756863,0.709804,0.576471,0.345098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10010,0.674510,0.670588,0.678431,0.686275,0.643137,0.733333,0.811765,0.823529,0.815686,0.807843,...,0.823529,0.823529,0.850980,0.866667,0.819608,0.725490,0.733333,0.752941,0.752941,0.752941
10011,0.007843,0.133333,0.423529,0.454902,0.447059,0.466667,0.513725,0.545098,0.545098,0.568627,...,0.721569,0.678431,0.662745,0.658824,0.658824,0.560784,0.541176,0.325490,0.090196,0.011765
10012,0.478431,0.603922,0.635294,0.666667,0.701961,0.772549,0.784314,0.764706,0.792157,0.780392,...,0.874510,0.866667,0.843137,0.803922,0.733333,0.819608,0.776471,0.733333,0.643137,0.611765
10013,0.537255,0.560784,0.552941,0.545098,0.576471,0.596078,0.607843,0.596078,0.607843,0.623529,...,0.698039,0.674510,0.670588,0.686275,0.717647,0.694118,0.666667,0.662745,0.650980,0.666667


In [ ]:
X = np.array(X)

Binarizing the labels to 1-hot-encode the labels

In [ ]:
#binarize the labels
lb = LabelBinarizer()
y = lb.fit_transform(y)

In [ ]:
y

array([[0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1]])

In [ ]:
print(X.shape)
print(y.shape)

(10015, 784)
(10015, 7)


Splitting into train and test

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size=0.2, random_state=1)

In [ ]:
def create_clients(X_train, Y_train, num_clients=10, initial='clients'):
    #create a list of client names
    client_names = ['{}_{}'.format(initial, i+1) for i in range(num_clients)]

    #randomize the data
    data = list(zip(X_train, Y_train))
    random.shuffle(data)

    #shard data and place at each client
    size = len(data)//num_clients
    shards = [data[i:i + size] for i in range(0, size*num_clients, size)]

    #number of clients must equal number of shards
    assert(len(shards) == len(client_names))

    return {client_names[i] : shards[i] for i in range(len(client_names))} 

In [ ]:
#create clients
clients = create_clients(X_train, Y_train, num_clients=10, initial='client')

In [ ]:
def batch_data(data_shard, bs=32):
    #seperate shard into data and labels lists
    data, label = zip(*data_shard)
    dataset = tf.data.Dataset.from_tensor_slices((list(data), list(label)))
    return dataset.shuffle(len(label)).batch(bs)

In [ ]:
#process and batch the training data for each client
clients_batched = dict()
for (client_name, data) in clients.items():
    clients_batched[client_name] = batch_data(data)
    
#process and batch the test set  
test_batched = tf.data.Dataset.from_tensor_slices((X_test, Y_test)).batch(len(Y_test))

In [ ]:
class SimpleMLP:
    @staticmethod
    def build(shape, classes):
        model = Sequential()
        model.add(Dense(200, input_shape=(784,)))
        model.add(Activation("relu"))
        model.add(Dense(200))
        model.add(Activation("relu"))
        model.add(Dense(7))
        model.add(Activation("softmax"))
        return model

In [ ]:
learning_rate = 0.01 
comms_round = 100
loss='categorical_crossentropy'
metrics = ['accuracy']
optimizer = SGD(learning_rate=learning_rate, 
                decay=learning_rate/ comms_round, 
                momentum=0.9
               )                      

In [ ]:
def weight_scalling_factor(clients_trn_data, client_name):
    client_names = list(clients_trn_data.keys())
    #get the bs
    bs = list(clients_trn_data[client_name])[0][0].shape[0]
    #first calculate the total training data points across clinets
    global_count = sum([tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy() for client_name in client_names])*bs
    # get the total number of data points held by a client
    local_count = tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy()*bs
    return local_count/global_count


def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final

In [ ]:
def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
        
    return avg_grad


def test_model(X_test, Y_test,  model, comm_round):
    cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    #logits = model.predict(X_test, batch_size=100)
    logits = model.predict(X_test)
    loss = cce(Y_test, logits)
    acc = accuracy_score(tf.argmax(logits, axis=1), tf.argmax(Y_test, axis=1))
    print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))
    return acc, loss

In [ ]:
def fl_model(clients_batched, test_batched, comms_round, loss, optimizer, metrics):
    #initialize global model
    smlp_global = SimpleMLP()
    global_model = smlp_global.build(784,7)

    #commence global training loop
    for comm_round in range(comms_round):
            
        # get the global model's weights - will serve as the initial weights for all local models
        global_weights = global_model.get_weights()
    
        #initial list to collect local model weights after scalling
        scaled_local_weight_list = list()

        #randomize client data - using keys
        client_names= list(clients_batched.keys())
        random.shuffle(client_names)
    
        #loop through each client and create new local model
        for client in client_names:
            smlp_local = SimpleMLP()
            local_model = smlp_local.build(784, 7)
            local_model.compile(loss=loss, 
                          optimizer=optimizer, 
                          metrics=metrics)
        
            #set local model weight to the weight of the global model
            local_model.set_weights(global_weights)
        
            #fit local model with client's data
            local_model.fit(clients_batched[client], epochs=1, verbose=0)
        
            #scale the model weights and add to list
            scaling_factor = weight_scalling_factor(clients_batched, client)
            scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
            scaled_local_weight_list.append(scaled_weights)
        
            #clear session to free memory after each communication round
            K.clear_session()
        
        #to get the average over all the local model, we simply take the sum of the scaled weights
        average_weights = sum_scaled_weights(scaled_local_weight_list)
    
        #update global model 
        global_model.set_weights(average_weights)

        #test global model and print out metrics after each communications round
        for(X_test, Y_test) in test_batched:
            global_acc, global_loss = test_model(X_test, Y_test, global_model, comm_round)
            
            # Make predictions on the test set
            y_pred = global_model.predict(X_test)
            y_pred = np.argmax(y_pred, axis=1)
            
            # Get the true classes
            true_classes = np.argmax(Y_test, axis = 1)
      
             # Print the classification report
            report = classification_report(true_classes, y_pred)
            from sklearn.metrics import confusion_matrix
            confusion_matrix = confusion_matrix(true_classes, y_pred)
            if comm_round == 99:
              print(confusion_matrix)
              plt.show()
              print(report)

In [ ]:
fl_model(clients_batched, test_batched, comms_round, loss, optimizer, metrics)

63/63 [==============================] - 0s 2ms/step
comm_round: 0 | global_acc: 66.251% | global_loss: 1.6503382921218872
63/63 [==============================] - 0s 2ms/step


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


63/63 [==============================] - 0s 5ms/step
comm_round: 1 | global_acc: 66.251% | global_loss: 1.6510295867919922
63/63 [==============================] - 0s 3ms/step


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


63/63 [==============================] - 0s 2ms/step
comm_round: 2 | global_acc: 66.251% | global_loss: 1.6228222846984863
63/63 [==============================] - 0s 2ms/step


...

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


63/63 [==============================] - 0s 2ms/step
comm_round: 98 | global_acc: 67.549% | global_loss: 1.6161102056503296
63/63 [==============================] - 0s 2ms/step


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


63/63 [==============================] - 0s 2ms/step
comm_round: 99 | global_acc: 67.299% | global_loss: 1.5771993398666382
63/63 [==============================] - 0s 2ms/step
[[   0    1   10    0   50    0    0]
 [   0    0    1    0   94    0    1]
 [   0    0   17    0  208    0    3]
 [   0    0    8    0   29    0    0]
 [   0    0    8    0 1312    0    7]
 [   0    0    1    0   29    0    2]
 [   0    0    7    0  196    0   19]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        61
           1       0.00      0.00      0.00        96
           2       0.33      0.07      0.12       228
           3       0.00      0.00      0.00        37
           4       0.68      0.99      0.81      1327
           5       0.00      0.00      0.00        32
           6       0.59      0.09      0.15       222

    accuracy                           0.67      2003
   macro avg       0.23      0.16      0.15      2003
weighted avg  

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
def test_model(X_test, Y_test,  model, comm_round):
    cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    #logits = model.predict(X_test, batch_size=100)
    logits = model.predict(X_test)
    loss = cce(Y_test, logits)
    acc = accuracy_score(tf.argmax(logits, axis=1), tf.argmax(Y_test, axis=1))
    print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))
    return acc, loss

In [ ]:
SGD_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).shuffle(len(Y_train)).batch(320)
smlp_SGD = SimpleMLP()
SGD_model = smlp_SGD.build(784, 7) 

SGD_model.compile(loss=loss, 
              optimizer=optimizer, 
              metrics=metrics)

# fit the SGD training data to model
_ = SGD_model.fit(SGD_dataset, epochs=100, verbose=0)

#test the SGD global model and print out metrics
for(X_test, Y_test) in test_batched:
        SGD_acc, SGD_loss = test_model(X_test, Y_test, SGD_model, 1)
# Make predictions on the test set
y_pred = SGD_model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
            
# Get the true classes
true_classes = np.argmax(Y_test, axis = 1)
      
# Print the classification report and the confusion matrix
report = classification_report(true_classes, y_pred)
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(true_classes, y_pred)
print(confusion_matrix)
plt.show()
print(report)

63/63 [==============================] - 0s 2ms/step
comm_round: 1 | global_acc: 67.099% | global_loss: 1.556148648262024
63/63 [==============================] - 0s 2ms/step
[[   0    2    7    0   52    0    0]
 [   0    1    1    0   93    0    1]
 [   0    1   13    0  209    0    5]
 [   0    1    6    0   30    0    0]
 [   0    0    7    0 1312    0    8]
 [   0    0    1    0   29    0    2]
 [   0    0    6    0  198    0   18]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        61
           1       0.20      0.01      0.02        96
           2       0.32      0.06      0.10       228
           3       0.00      0.00      0.00        37
           4       0.68      0.99      0.81      1327
           5       0.00      0.00      0.00        32
           6       0.53      0.08      0.14       222

    accuracy                           0.67      2003
   macro avg       0.25      0.16      0.15      2003
weighted avg    

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Label-flipping

In [ ]:
## Pseudocode for flipping percent p of labels.

# Store all possible labels in list all_labels.
# k =  (total samples of client * p)/100     

# For one particular client dataset- do following.
# Get k random indexes of dataset and store in a list all_indexes (list contains k random numbers from 0 - number of samples for that client).
# For each value in the all_indexes list obtained above - do following
# Replace label with some random label from list all_labels

In [ ]:
client_1,label_1=zip(*clients['client_1'])

In [ ]:
# before replacing labels for label_1
for i, label in enumerate(label_1):
    print(i, label)

0 [0 0 0 0 1 0 0]
1 [0 0 0 0 1 0 0]
2 [0 0 0 0 1 0 0]
3 [0 0 1 0 0 0 0]
4 [0 0 0 0 1 0 0]
5 [0 0 0 0 0 0 1]
6 [0 0 0 0 1 0 0]
7 [0 0 0 0 0 1 0]
8 [0 0 1 0 0 0 0]
9 [0 0 0 0 1 0 0]
10 [0 0 0 0 1 0 0]
11 [0 0 0 0 0 1 0]
12 [0 0 0 0 0 0 1]
13 [0 0 0 0 1 0 0]
14 [0 0 0 0 1 0 0]
15 [0 0 0 0 1 0 0]
16 [0 1 0 0 0 0 0]
17 [0 0 1 0 0 0 0]
18 [0 0 0 0 0 0 1]
19 [0 0 0 0 1 0 0]
20 [0 0 0 0 0 0 1]
21 [1 0 0 0 0 0 0]
22 [0 0 0 0 1 0 0]
23 [0 0 0 0 1 0 0]
24 [0 0 0 0 1 0 0]
25 [0 1 0 0 0 0 0]
26 [0 0 0 0 1 0 0]
27 [0 0 0 0 1 0 0]
28 [0 0 0 0 0 0 1]
29 [0 0 0 0 0 0 1]
30 [0 0 0 0 1 0 0]
31 [0 0 0 0 1 0 0]
32 [0 0 0 0 1 0 0]
33 [0 0 0 0 1 0 0]
34 [0 0 0 0 1 0 0]
35 [0 1 0 0 0 0 0]
36 [0 0 0 0 1 0 0]
37 [0 0 1 0 0 0 0]
38 [0 0 0 0 1 0 0]
39 [0 0 0 0 1 0 0]
40 [0 0 1 0 0 0 0]
41 [0 0 0 0 0 0 1]
42 [0 0 0 0 1 0 0]
43 [0 0 0 0 1 0 0]
44 [0 0 0 0 0 0 1]
45 [0 0 0 0 1 0 0]
46 [0 0 0 0 1 0 0]
47 [0 0 0 0 1 0 0]
48 [0 0 0 0 1 0 0]
49 [0 0 0 0 0 0 1]
50 [0 0 0 0 1 0 0]
51 [0 1 0 0 0 0 0]
52 [0 0 0 0 1 0 0]
53 

In [ ]:
# getting all labels under all_labels variable
all_labels = ()
clients_1 = ['client_1','client_2','client_3','client_4','client_5','client_6','client_7','client_8','client_9','client_10']
for client in range(len(clients_1)):
    name = clients_1[client]
    image,label = zip(*clients[name])
    all_labels=all_labels+(label)
print(list(all_labels))


[array([0, 0, 0, 0, 1, 0, 0]), array([0, 0, 0, 0, 1, 0, 0]), array([0, 0, 0, 0, 1, 0, 0]), array([0, 0, 1, 0, 0, 0, 0]), array([0, 0, 0, 0, 1, 0, 0]), array([0, 0, 0, 0, 0, 0, 1]), array([0, 0, 0, 0, 1, 0, 0]), array([0, 0, 0, 0, 0, 1, 0]), array([0, 0, 1, 0, 0, 0, 0]), array([0, 0, 0, 0, 1, 0, 0]), array([0, 0, 0, 0, 1, 0, 0]), array([0, 0, 0, 0, 0, 1, 0]), array([0, 0, 0, 0, 0, 0, 1]), array([0, 0, 0, 0, 1, 0, 0]), array([0, 0, 0, 0, 1, 0, 0]), array([0, 0, 0, 0, 1, 0, 0]), array([0, 1, 0, 0, 0, 0, 0]), array([0, 0, 1, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 1]), array([0, 0, 0, 0, 1, 0, 0]), array([0, 0, 0, 0, 0, 0, 1]), array([1, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 1, 0, 0]), array([0, 0, 0, 0, 1, 0, 0]), array([0, 0, 0, 0, 1, 0, 0]), array([0, 1, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 1, 0, 0]), array([0, 0, 0, 0, 1, 0, 0]), array([0, 0, 0, 0, 0, 0, 1]), array([0, 0, 0, 0, 0, 0, 1]), array([0, 0, 0, 0, 1, 0, 0]), array([0, 0, 0, 0, 1, 0, 0]), array([0, 0, 0, 0, 1, 0, 0]), array([0,

In [ ]:
len(all_labels)

8010

In [ ]:
# finding the k
p=20
k= len(clients['client_1'])*p//100
print(k)

160


In [ ]:
# unzipped the client_1 into images and labels
len_of_one_client_dataset = len(client_1)
print(len_of_one_client_dataset)

801


In [ ]:
all_indexes = np.random.randint(0, 801, k)

print(all_indexes)

[513 360 798 369 485 726 443 554 677 591 252 653 617  64 778 320 512 207
 781 744 391  86 447 128 286 345  63 594 408 526 609 308 505  85 295 427
 120 263 348 121 561 581  60 750 351 344 257 232 482 473 624 681 610 629
 396 137 578 189 722 213 388 586 509 313 389 315 557 528 407  73 106 424
 641 690 633  30 536 730 452 790 523 647 613 709 785 202 377 762 531 519
 476 789 707 281 362 191 384 379 447 270 511  15 292 105 512  49 282 362
 725 304 334 773 469 731 715 228 301 548  82 359 708 519 331 521  48 447
 693 212 394 604 149 622 777 590 333 500 752 288 641 137 409 380 777 199
 108 452 163 647 727 591  18 759 106 129 373 426  31 326 206 134]


In [ ]:
label_1 = list(label_1)

# Replace the label at each index in all_indexes with a random label from all_labels
for index in all_indexes:
    label_1[index] = random.choice(all_labels)

In [ ]:
print(label_1)

[array([0, 0, 0, 0, 1, 0, 0]), array([0, 0, 0, 0, 1, 0, 0]), array([0, 0, 0, 0, 1, 0, 0]), array([0, 0, 1, 0, 0, 0, 0]), array([0, 0, 0, 0, 1, 0, 0]), array([0, 0, 0, 0, 0, 0, 1]), array([0, 0, 0, 0, 1, 0, 0]), array([0, 0, 0, 0, 0, 1, 0]), array([0, 0, 1, 0, 0, 0, 0]), array([0, 0, 0, 0, 1, 0, 0]), array([0, 0, 0, 0, 1, 0, 0]), array([0, 0, 0, 0, 0, 1, 0]), array([0, 0, 0, 0, 0, 0, 1]), array([0, 0, 0, 0, 1, 0, 0]), array([0, 0, 0, 0, 1, 0, 0]), array([0, 0, 0, 0, 1, 0, 0]), array([0, 1, 0, 0, 0, 0, 0]), array([0, 0, 1, 0, 0, 0, 0]), array([0, 0, 1, 0, 0, 0, 0]), array([0, 0, 0, 0, 1, 0, 0]), array([0, 0, 0, 0, 0, 0, 1]), array([1, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 1, 0, 0]), array([0, 0, 0, 0, 1, 0, 0]), array([0, 0, 0, 0, 1, 0, 0]), array([0, 1, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 1, 0, 0]), array([0, 0, 0, 0, 1, 0, 0]), array([0, 0, 0, 0, 0, 0, 1]), array([0, 0, 0, 0, 0, 0, 1]), array([0, 0, 0, 0, 1, 0, 0]), array([0, 1, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 1, 0, 0]), array([0,

In [ ]:
# cross checking from all_labels
all_labels[513]

array([0, 0, 1, 0, 0, 0, 0])

In [ ]:
type(label_1)

list

In [ ]:
# assigning the replaced labels to particular client
# Replace the labels in the clients dictionary
clients["client_1"] = list(zip(client_1, label_1))

In [ ]:
# cross checking if it is assigned or not
clients['client_1'][513]

(array([0.00392157, 0.02352941, 0.04705882, 0.04313725, 0.05490196,
        0.20392157, 0.29803922, 0.20784314, 0.09411765, 0.0745098 ,
        0.07843137, 0.08235294, 0.08235294, 0.06666667, 0.05098039,
        0.05098039, 0.06666667, 0.09019608, 0.11764706, 0.08627451,
        0.06666667, 0.08235294, 0.10588235, 0.14901961, 0.19215686,
        0.19215686, 0.15686275, 0.09411765, 0.01176471, 0.03137255,
        0.04313725, 0.02745098, 0.2       , 0.40784314, 0.45098039,
        0.4627451 , 0.38431373, 0.24313725, 0.1372549 , 0.0745098 ,
        0.07843137, 0.09019608, 0.0745098 , 0.0627451 , 0.06666667,
        0.07058824, 0.09803922, 0.14509804, 0.10588235, 0.0745098 ,
        0.09803922, 0.12156863, 0.16470588, 0.19215686, 0.17254902,
        0.12156863, 0.01960784, 0.04313725, 0.02745098, 0.11372549,
        0.43137255, 0.5254902 , 0.51764706, 0.55294118, 0.54117647,
        0.52941176, 0.4745098 , 0.3372549 , 0.19607843, 0.09019608,
        0.08627451, 0.08235294, 0.0745098 , 0.08

In [ ]:
# cross checking after replacing for label_1
for i in all_indexes:
    print(label_1[i])

[0 0 0 0 1 0 0]
[1 0 0 0 0 0 0]
[0 0 0 0 1 0 0]
[0 0 1 0 0 0 0]
[0 0 0 0 1 0 0]
[0 0 0 0 1 0 0]
[0 0 0 0 1 0 0]
[0 0 0 0 1 0 0]
[0 0 1 0 0 0 0]
[0 0 0 0 1 0 0]
[0 0 0 0 1 0 0]
[0 1 0 0 0 0 0]
[0 0 0 0 1 0 0]
[0 0 0 0 1 0 0]
[0 0 0 0 1 0 0]
[0 0 0 0 0 0 1]
[1 0 0 0 0 0 0]
[0 0 0 0 1 0 0]
[0 0 0 0 1 0 0]
[0 0 0 0 1 0 0]
[0 1 0 0 0 0 0]
[1 0 0 0 0 0 0]
[0 1 0 0 0 0 0]
[0 0 0 1 0 0 0]
[0 0 0 0 0 0 1]
[0 0 0 0 1 0 0]
[0 0 1 0 0 0 0]
[0 0 1 0 0 0 0]
[0 0 0 0 1 0 0]
[0 0 0 0 1 0 0]
[0 0 0 0 1 0 0]
[0 0 1 0 0 0 0]
[0 0 0 0 1 0 0]
[0 0 0 0 1 0 0]
[0 1 0 0 0 0 0]
[0 0 0 0 1 0 0]
[0 0 0 0 1 0 0]
[0 0 0 0 0 0 1]
[0 0 0 0 0 0 1]
[0 0 0 0 1 0 0]
[0 0 0 0 0 0 1]
[0 0 1 0 0 0 0]
[0 0 0 0 1 0 0]
[0 0 0 0 1 0 0]
[0 0 0 0 0 0 1]
[0 0 0 0 1 0 0]
[0 0 0 0 1 0 0]
[0 0 0 0 1 0 0]
[0 0 1 0 0 0 0]
[0 0 0 0 1 0 0]
[0 0 0 0 1 0 0]
[0 0 0 0 1 0 0]
[0 0 0 0 1 0 0]
[0 0 0 0 0 0 1]
[0 0 0 0 1 0 0]
[0 0 0 0 1 0 0]
[0 0 0 0 1 0 0]
[0 0 1 0 0 0 0]
[0 0 0 0 1 0 0]
[0 0 0 0 1 0 0]
[0 0 0 0 1 0 0]
[0 0 1 0 0 0 0]
[0 0 0 0

In [ ]:
fl_model(clients_batched, test_batched, comms_round, loss, optimizer, metrics)

63/63 [==============================] - 1s 2ms/step
comm_round: 0 | global_acc: 66.251% | global_loss: 1.6204502582550049
63/63 [==============================] - 0s 2ms/step


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


63/63 [==============================] - 0s 2ms/step
comm_round: 1 | global_acc: 66.251% | global_loss: 1.6243340969085693
63/63 [==============================] - 0s 2ms/step


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


63/63 [==============================] - 0s 2ms/step
comm_round: 2 | global_acc: 66.251% | global_loss: 1.6312410831451416
63/63 [==============================] - 0s 2ms/step


...

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


63/63 [==============================] - 0s 2ms/step
comm_round: 98 | global_acc: 67.149% | global_loss: 1.605159044265747
63/63 [==============================] - 0s 3ms/step


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


63/63 [==============================] - 0s 2ms/step
comm_round: 99 | global_acc: 67.199% | global_loss: 1.5937548875808716
63/63 [==============================] - 0s 2ms/step
[[   0    2    8    0   51    0    0]
 [   0    0    2    0   93    0    1]
 [   0    1   13    0  209    0    5]
 [   0    0    6    0   31    0    0]
 [   0    1    5    0 1315    0    6]
 [   0    0    1    0   30    0    1]
 [   0    0    7    0  197    0   18]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        61
           1       0.00      0.00      0.00        96
           2       0.31      0.06      0.10       228
           3       0.00      0.00      0.00        37
           4       0.68      0.99      0.81      1327
           5       0.00      0.00      0.00        32
           6       0.58      0.08      0.14       222

    accuracy                           0.67      2003
   macro avg       0.22      0.16      0.15      2003
weighted avg  

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
SGD_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).shuffle(len(Y_train)).batch(320)
smlp_SGD = SimpleMLP()
SGD_model = smlp_SGD.build(784, 7) 

SGD_model.compile(loss=loss, 
              optimizer=optimizer, 
              metrics=metrics)

# fit the SGD training data to model
_ = SGD_model.fit(SGD_dataset, epochs=100, verbose=0)

#test the SGD global model and print out metrics
for(X_test, Y_test) in test_batched:
        SGD_acc, SGD_loss = test_model(X_test, Y_test, SGD_model, 1)

        # Make predictions on the test set
        y_pred = SGD_model.predict(X_test)
        y_pred = np.argmax(y_pred, axis=1)
            
        # Get the true classes
        true_classes = np.argmax(Y_test, axis = 1)
      
        # Print the classification report and the confusion matrix
        report = classification_report(true_classes, y_pred)
        from sklearn.metrics import confusion_matrix
        confusion_matrix = confusion_matrix(true_classes, y_pred)
        print(confusion_matrix)
        plt.show()
        print(report)

63/63 [==============================] - 0s 2ms/step
comm_round: 1 | global_acc: 66.950% | global_loss: 1.5530188083648682
63/63 [==============================] - 0s 2ms/step
[[   0    0    5    0   56    0    0]
 [   0    0    1    0   94    0    1]
 [   0    0    5    0  217    0    6]
 [   0    0    4    0   33    0    0]
 [   0    0    4    0 1318    0    5]
 [   0    0    1    0   30    0    1]
 [   0    0    2    0  202    0   18]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        61
           1       0.00      0.00      0.00        96
           2       0.23      0.02      0.04       228
           3       0.00      0.00      0.00        37
           4       0.68      0.99      0.80      1327
           5       0.00      0.00      0.00        32
           6       0.58      0.08      0.14       222

    accuracy                           0.67      2003
   macro avg       0.21      0.16      0.14      2003
weighted avg   

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
